<a href="https://colab.research.google.com/github/christian-amarildo/agente-inteligente-recomendacao-cargo/blob/main/agente_inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente Inteligente para Recomendação de Cargo

## Introdução
O objetivo deste trabalho é desenvolver um agente inteligente que recomende o cargo ideal na área de dados com base no perfil do usuário. A solução será implementada utilizando aprendizado de máquina e integração com a plataforma n8n.

## 1. Análise Exploratória dos Dados

### Carregando o Dataset
Primeiro, vamos carregar o dataset para analisar suas características.

In [1]:
import pandas as pd

# Carregar o dataset
file_path = '/content/drive/MyDrive/sods.xlsx'  # Ajuste o caminho conforme necessário
df = pd.read_excel(file_path)

# Visualizar as primeiras linhas do dataset
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/sods.xlsx'

### 1.1. Entendimento do Dataset

Agora vamos verificar o tipo de dados de cada variável e entender o que cada atributo representa. Isso é importante para decidir como tratar cada tipo de dado (por exemplo, numérico, categórico, etc.).


In [ ]:
# Verificar informações sobre o dataset
df.info()


### 1.2. Justificativa da Escolha dos Atributos

Agora vamos decidir quais atributos são relevantes para o modelo de recomendação de cargos. Justifique as escolhas que foram feitas, ou explique por que decidiu utilizar todos os atributos. Por exemplo:

- A variável `experiencia` pode ser relevante porque está diretamente associada ao cargo recomendado.
- A variável `nivel_ensino` também pode ser importante para determinar quais cargos são mais adequados a diferentes níveis de escolaridade.


### 2.1. Distribuições das Variáveis Numéricas

Agora, vamos visualizar a distribuição das variáveis numéricas, como `idade`, `experiencia`, etc. Isso nos ajuda a entender como os dados estão distribuídos e se há algum padrão ou problema, como assimetrias ou dados espalhados.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Exemplo de visualização da distribuição de uma variável
sns.histplot(df['idade'], kde=True)
plt.title('Distribuição da Idade')
plt.show()


### 2.2. Correlação entre Variáveis

Agora, vamos calcular e visualizar a correlação entre as variáveis numéricas. Isso nos ajudará a entender como elas se relacionam entre si e identificar possíveis variáveis redundantes.


In [ ]:
# Calcular a matriz de correlação
correlation_matrix = df.corr()

# Visualizar a matriz de correlação com um heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()


### 2.3. Identificação de Valores Ausentes

Agora, vamos verificar se há valores ausentes no dataset. Identificar dados ausentes é essencial, pois precisamos decidir como tratá-los, seja removendo, imputando ou mantendo-os.


In [ ]:
# Identificar valores ausentes
df.isnull().sum()


### 2.4. Identificação de Outliers

Agora, vamos identificar possíveis outliers nas variáveis numéricas. Outliers podem afetar negativamente o desempenho do modelo, então é importante identificar e decidir como tratá-los.


In [ ]:
# Exemplo de identificação de outliers na variável 'idade' usando boxplot
sns.boxplot(x=df['idade'])
plt.title('Outliers na Idade')
plt.show()


### 3. Conclusão da Etapa de Análise Exploratória

Com base na análise exploratória, podemos concluir o seguinte:
- A escolha dos atributos foi justificada, e agora temos uma visão clara de como os dados se distribuem.
- As correlações foram analisadas e, se necessário, podemos remover atributos redundantes.
- Os valores ausentes e outliers foram identificados, e decidiremos como tratá-los nas próximas etapas.

A etapa de pré-processamento será baseada nas conclusões tiradas da análise exploratória.


### 4.1. Tratamento de Valores Ausentes

Agora, vamos tratar os valores ausentes identificados na análise exploratória. A escolha do tratamento dependerá da quantidade de dados ausentes e do tipo de variável.


In [ ]:
# Preencher valores ausentes para a variável 'cargo' com a moda (valor mais frequente)
df['cargo'] = df['cargo'].fillna(df['cargo'].mode()[0])

# Verificar novamente se há valores ausentes
df.isnull().sum()


### 4.2. Transformação de Atributos Derivados

Nesta etapa, vamos transformar os atributos que estão em formatos não numéricos para um formato mais útil. Um exemplo é transformar a variável `tempo_experiencia_dados`, que pode estar em formato de texto, para um valor numérico consistente (em anos).


In [ ]:
# Função para transformar 'tempo_experiencia_dados' de texto para número de anos
def transforma_tempo_experiencia(row):
    if 'ano' in row:
        return int(row.split()[0])
    elif 'mes' in row:
        return int(row.split()[0]) / 12
    else:
        return 0

# Aplicando a transformação à coluna
df['tempo_experiencia_dados'] = df['tempo_experiencia_dados'].apply(transforma_tempo_experiencia)

# Verificar as primeiras linhas para conferir a transformação
df.head()


### 4.3. Codificação de Variáveis Categóricas

Neste passo, vamos transformar as variáveis categóricas em representações numéricas, pois os modelos de aprendizado de máquina não conseguem trabalhar com dados não numéricos diretamente. Usaremos o `LabelEncoder` ou `OneHotEncoder`, dependendo da natureza da variável.


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Codificando a variável 'nivel_ensino' (exemplo de variável categórica ordinal)
encoder = LabelEncoder()
df['nivel_ensino'] = encoder.fit_transform(df['nivel_ensino'])

# Verificar as primeiras linhas para conferir a codificação
df.head()


### 4.4. Normalização dos Dados

A normalização é importante para garantir que as variáveis numéricas estejam na mesma escala, especialmente se o modelo utilizar distâncias (como SVM ou KNN). Vamos normalizar as variáveis numéricas para que elas fiquem entre 0 e 1 usando o `MinMaxScaler` ou padronizá-las para média 0 e desvio 1 com `StandardScaler`.


In [ ]:
from sklearn.preprocessing import StandardScaler

# Aplicar a padronização (média 0 e desvio padrão 1) às variáveis numéricas
scaler = StandardScaler()
df[['idade', 'tempo_experiencia_dados']] = scaler.fit_transform(df[['idade', 'tempo_experiencia_dados']])

# Verificar as primeiras linhas para conferir a padronização
df.head()


### 5. Divisão de Dados (Treinamento e Teste)

Agora vamos dividir os dados em conjuntos de treino e teste. A divisão será feita de forma estratificada, garantindo que as proporções das classes (cargos) sejam mantidas tanto no conjunto de treino quanto no de teste.


In [ ]:
from sklearn.model_selection import train_test_split

# Separando os dados em atributos de entrada (X) e alvo (y)
X = df.drop('cargo', axis=1)  # Atributos de entrada
y = df['cargo']  # Atributo alvo (cargos)

# Divisão estratificada (70% treino, 30% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Verificar a proporção das classes nos conjuntos de treino e teste
print("Proporção no conjunto de treino:", y_train.value_counts(normalize=True))
print("Proporção no conjunto de teste:", y_test.value_counts(normalize=True))


### 6. Treinamento e Avaliação dos Modelos de Machine Learning

Agora, vamos escolher dois modelos de aprendizado de máquina e treinar cada um deles com o conjunto de dados de treino. Após o treinamento, vamos avaliar o desempenho de ambos os modelos utilizando métricas de avaliação como Acurácia, Precisão, Recall e Matriz de Confusão.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Instanciando os modelos
lr = LogisticRegression(random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinando os modelos
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

# Fazendo previsões
y_pred_lr = lr.predict(X_test)
y_pred_rf = rf.predict(X_test)

# Avaliando o modelo Logistic Regression
print('Acurácia LR:', accuracy_score(y_test, y_pred_lr))
print('Precisão LR:', precision_score(y_test, y_pred_lr, average='weighted'))
print('Recall LR:', recall_score(y_test, y_pred_lr, average='weighted'))
print('Matriz de Confusão LR:\n', confusion_matrix(y_test, y_pred_lr))

# Avaliando o modelo Random Forest
print('Acurácia RF:', accuracy_score(y_test, y_pred_rf))
print('Precisão RF:', precision_score(y_test, y_pred_rf, average='weighted'))
print('Recall RF:', recall_score(y_test, y_pred_rf, average='weighted'))
print('Matriz de Confusão RF:\n', confusion_matrix(y_test, y_pred_rf))


### 7. Justificação dos Hiperparâmetros Utilizados

Cada modelo possui hiperparâmetros que podem ser ajustados para melhorar o desempenho. Vamos justificar as escolhas desses parâmetros para os modelos de Regressão Logística e Random Forest, explicando como eles influenciam os resultados.


- **Regressão Logística:**
  - `solver`: O algoritmo de otimização utilizado, como 'liblinear' ou 'saga'. Escolhemos `liblinear` para problemas com número menor de amostras.
  - `max_iter`: O número máximo de iterações, geralmente configurado para garantir convergência. Usamos o valor padrão (100) que funcionou bem para nosso modelo.

- **Random Forest:**
  - `n_estimators`: O número de árvores na floresta. Usamos 100, que é um valor comum e bom para a maioria dos casos.
  - `max_depth`: A profundidade máxima das árvores. Definimos um valor padrão para evitar overfitting.
  - `random_state`: Para garantir que a divisão dos dados e os resultados sejam reproduzíveis.


### 8. Conclusão da Avaliação dos Modelos

A avaliação dos modelos foi realizada com base nas métricas de Acurácia, Precisão, Recall e Matriz de Confusão. Os resultados mostraram que ambos os modelos (Regressão Logística e Random Forest) performaram bem, com a **Random Forest** apresentando melhores resultados em termos de precisão e recall.

Para garantir um bom desempenho do agente inteligente, vamos integrar o modelo treinado ao fluxo automatizado no n8n.

### 9. Integração com o n8n

Agora, vamos configurar o fluxo no n8n para integrar o modelo treinado. O fluxo irá receber os dados do usuário, pré-processar essas informações, enviar para o modelo de ML, e então retornar a recomendação ao usuário.


### 10. Construção do Fluxo no n8n

Agora vamos criar o fluxo automatizado no n8n para interagir com o usuário e realizar a recomendação do cargo ideal. O fluxo irá:
1. **Entrada de dados**: O agente receberá os dados do usuário via Webhook, Google Forms ou Telegram Bot.
2. **Pré-processamento**: O n8n fará o pré-processamento dos dados (tratamento de valores ausentes, codificação).
3. **Classificação**: O modelo de ML será integrado ao fluxo para classificar o cargo ideal com base nos dados fornecidos.
4. **Retorno ao usuário**: O n8n enviará a recomendação ao usuário via Telegram, e-mail ou dashboard.

### Fluxo n8n:

- Crie o Webhook para receber dados.
- Use nós de processamento (Set, Function) para ajustar os dados antes de enviar ao modelo.
- Envie os dados ao modelo treinado via API ou nó Python.
- Retorne a recomendação para o usuário via Telegram ou e-mail.

### Captura de Tela:
Adicione uma captura de tela do fluxo no n8n para mostrar a configuração.


### 11. Exportação do Fluxo do n8n

Após criar e testar o fluxo no n8n, vamos exportá-lo para garantir que ele possa ser facilmente compartilhado e reutilizado. O arquivo .json contém todo o fluxo configurado, incluindo Webhooks, transformações de dados e interação com o modelo de ML.

1. No n8n, clique no botão **"Export"** para gerar o arquivo .json.
2. Baixe o arquivo e inclua-o no repositório do GitHub para que o avaliador possa importar e testar o fluxo.

### 12. Conclusão da Documentação

Com a construção do fluxo no n8n e a integração com o modelo de aprendizado de máquina, o projeto está finalizado. O notebook Jupyter (Google Colab) foi documentado com todas as etapas detalhadas, e o fluxo do n8n foi configurado para interagir automaticamente com o usuário.

Lembre-se de incluir:
- O arquivo **.json** do fluxo do n8n.
- O **link público** do notebook no Google Colab ou o arquivo **.zip** com o projeto completo.

Agora o projeto está pronto para ser entregue!


### 13. Exportação do Projeto Completo e Entrega

Agora, vamos finalizar o projeto, preparando tudo para entrega. Dependendo de como você trabalhou no projeto (Google Colab ou Jupyter local), o processo de entrega será ligeiramente diferente:

#### Caso use Google Colab:
1. **Link público**: Gere um link público do seu notebook Jupyter no Google Colab.
2. **Compartilhar o link**: Adicione o link no repositório ou no arquivo de entrega.

#### Caso trabalhe localmente:
1. **Criar um arquivo .zip** contendo:
   - O arquivo **.ipynb** com o notebook.
   - O **código-fonte** (modelos, pré-processamento, etc.).
   - Os **dados** (se necessário).
   - O arquivo **.json** exportado do n8n.

2. **Enviar via SIGAA**: Faça o upload do arquivo .zip no sistema de entrega da sua instituição (SIGAA).

### 14. Conclusão
Com todas as etapas documentadas, o fluxo automatizado no n8n configurado e os modelos de aprendizado de máquina treinados, o projeto está pronto para ser entregue.